<a href="https://colab.research.google.com/github/z-guard/analysis/blob/main/notebooks/population_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人口マスタ

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [3]:
import os
import requests
import json
import pandas as pd
from urllib.parse import urlparse
import geocoder
import math

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [4]:
def get_file_from_url(url, save_path):
    res = requests.get(url)
    if res.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(res.content)
    else:
        raise Exception(f'status_code: {res.status_code} ({url})')

### 全区
- [平成27年国勢調査 東京都区市町村町丁別報告](https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd-15index.htm)

In [6]:
temp_dir = '/content/tmp'
os.makedirs(temp_dir, exist_ok=True)

In [7]:
# 人口・面積
zip_path_1 = os.path.join(temp_dir, '001.zip')
zip_url_1 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv01100000.zip'
get_file_from_url(zip_url_1, zip_path_1)

# 世帯数
zip_path_2 = os.path.join(temp_dir, '002.zip')
zip_url_2 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv02100000.zip'
get_file_from_url(zip_url_2, zip_path_2)

# 労働力人口・従業上の地位
zip_path_5 = os.path.join(temp_dir, '005.zip')
zip_url_5 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv05100000.zip'
get_file_from_url(zip_url_5, zip_path_5)

# 住居の種類
zip_path_6 = os.path.join(temp_dir, '006.zip')
zip_url_6 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv06100000.zip'
get_file_from_url(zip_url_6, zip_path_6)

# 住宅の建て方
zip_path_7 = os.path.join(temp_dir, '007.zip')
zip_url_7 = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv07100000.zip'
get_file_from_url(zip_url_7, zip_path_7)

In [8]:
!unzip -o -d $temp_dir $zip_path_1
!unzip -o -d $temp_dir $zip_path_2
!unzip -o -d $temp_dir $zip_path_5
!unzip -o -d $temp_dir $zip_path_6
!unzip -o -d $temp_dir $zip_path_7

Archive:  /content/tmp/001.zip
  inflating: /content/tmp/kd15zv01100000.csv  
Archive:  /content/tmp/002.zip
  inflating: /content/tmp/kd15zv02100000.csv  
Archive:  /content/tmp/005.zip
  inflating: /content/tmp/kd15zv05100000.csv  
Archive:  /content/tmp/006.zip
  inflating: /content/tmp/kd15zv06100000.csv  
Archive:  /content/tmp/007.zip
  inflating: /content/tmp/kd15zv07100000.csv  


## 人口・面積

In [9]:
csv_file_1 = 'kd15zv01100000.csv'
df_all_popdata = pd.read_csv(os.path.join(temp_dir, csv_file_1), dtype='str')
df_all_popdata = df_all_popdata[df_all_popdata['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_popdata.shape)

(19818, 160)


In [10]:
df_popdata = df_all_popdata[[
    '地域ＩＤ',
    '地域階層フラグ／丁目･小字相当',
    '表側表章地域（階層別）／区市町村階層以上の地域名称',
    '表側表章地域（階層別）／町・大字相当階層の地域名称',
    '表側表章地域（階層別）／丁目・小字等階層の地域名称',
    '表側／男女の別',
    '人口総数（人）',
    '面積（平方キロメートル） ',
    '人口密度（人／平方キロメートル） ',
    '年齢（５歳階級）／0～4歳（人）',
    '年齢（５歳階級）／5～9歳（人）',
    '年齢（５歳階級）／10～14歳（人）',
    '年齢（５歳階級）／15～19歳（人）',
    '年齢（５歳階級）／20～24歳（人）',
    '年齢（５歳階級）／25～29歳（人）',
    '年齢（５歳階級）／30～34歳（人）',
    '年齢（５歳階級）／35～39歳（人）',
    '年齢（５歳階級）／40～44歳（人）',
    '年齢（５歳階級）／45～49歳（人）',
    '年齢（５歳階級）／50～54歳（人）',
    '年齢（５歳階級）／55～59歳（人）',
    '年齢（５歳階級）／60～64歳（人）',
    '年齢（５歳階級）／65～69歳（人）',
    '年齢（５歳階級）／70～74歳（人）',
    '年齢（５歳階級）／75～79歳（人）',
    '年齢（５歳階級）／80～84歳（人）',
    '年齢（５歳階級）／85～89歳（人）',
    '年齢（５歳階級）／90～94歳（人）',
    '年齢（５歳階級）／95～99歳（人）',
    '年齢（５歳階級）／100歳以上（人）',
    '年齢（５歳階級）／年齢不詳（人）',
    '平均年齢（歳）',
    '（再掲）外国人（人） ',
]]
df_popdata.columns = [
    'town_id',
    '地域階層フラグ',
    '市区町村名',
    '町名',
    '町丁目',
    '男女',
    '人口総数（人）',
    '面積（km2）',
    '人口密度(人/km2)',
    '0〜4歳（人）',
    '5〜9歳（人）',
    '10〜14歳（人）',
    '15〜19歳（人）',
    '20〜24歳（人）',
    '25〜29歳（人）',
    '30〜34歳（人）',
    '35〜39歳（人）',
    '40〜44歳（人）',
    '45〜49歳（人）',
    '50〜54歳（人）',
    '55〜59歳（人）',
    '60〜64歳（人）',
    '65〜69歳（人）',
    '70〜74歳（人）',
    '75〜79歳（人）',
    '80〜84歳（人）',
    '85〜89歳（人）',
    '90〜94歳（人）',
    '95〜99歳（人）',
    '100歳以上（人）',
    '年齢不詳（人）',
    '平均年齢（歳）',
    '外国人（人）'
]
df_popdata = df_popdata.replace('X', 0).replace('-', 0)
df_popdata = df_popdata[
    ((df_popdata['地域階層フラグ'] == '●')|(df_popdata['地域階層フラグ'] == '○')) &
    (df_popdata['市区町村名'].str.endswith('区')) &
    (df_popdata['男女'] == '総数')
].drop(columns=['地域階層フラグ', '男女']).reset_index(drop=True)

df_popdata.loc[df_popdata['町丁目'].isna(), '町丁目'] = df_popdata['町名']
print(df_popdata.shape)

(3145, 31)


In [11]:
pop_cols = [col for col in df_popdata.columns if '（人）' in col]
for col in pop_cols:
    df_popdata[col] = df_popdata[col].astype(int)

df_popdata['面積（km2）'] = df_popdata['面積（km2）'].astype(float)
df_popdata['人口密度(人/km2)'] = df_popdata['人口密度(人/km2)'].astype(float)
df_popdata['平均年齢（歳）'] = df_popdata['平均年齢（歳）'].astype(float)

## 世帯数

In [12]:
csv_file_2 = 'kd15zv02100000.csv'
df_all_family = pd.read_csv(os.path.join(temp_dir, csv_file_2), dtype='str')
df_all_family = df_all_family[df_all_family['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_family.shape)

(6606, 41)


In [13]:
df_family = df_all_family[[
    '地域ＩＤ',
    '世帯総数／一般世帯数／総数（世帯）',
    '世帯総数／一般世帯数／世帯人員が1人（世帯）',
    '世帯総数／一般世帯数／世帯人員が2人（世帯）',
    '世帯総数／一般世帯数／世帯人員が3人（世帯）',
    '世帯総数／一般世帯数／世帯人員が4人（世帯）',
    '世帯総数／一般世帯数／世帯人員が5人（世帯）',
    '世帯総数／一般世帯数／世帯人員が6人（世帯）',
    '世帯総数／一般世帯数／世帯人員が7人以上（世帯）',
    '世帯総数／一般世帯人員（人）',
    '世帯総数／一般世帯の1世帯当たり人員（人）',
]].copy()
df_family.columns = [
    'town_id',
    '世帯総数',
    '1人世帯',
    '2人世帯',
    '3人世帯',
    '4人世帯',
    '5人世帯',
    '6人世帯',
    '7人以上世帯',
    '世帯人員（人）',
    '1世帯当たり人員（人）',
]
df_family = df_family.replace('X', 0).replace('-', 0)
for col in df_family.columns:
    if col == 'town_id':
        continue
    elif col == '1世帯当たり人員（人）':
        df_family[col] = df_family[col].astype(float)
    else:
        df_family[col] = df_family[col].astype(int)

df_family['5人以上世帯'] = df_family['5人世帯'] + df_family['6人世帯'] + df_family['7人以上世帯']
df_family = df_family[[
    'town_id', '世帯総数', '1人世帯', '2人世帯', '3人世帯', '4人世帯', '5人以上世帯', '世帯人員（人）', '1世帯当たり人員（人）',
]].reset_index(drop=True)
print(df_family.shape)
assert df_family['town_id'].nunique() == len(df_family)

(6606, 9)


## 労働力人口・従業上の地位

In [14]:
csv_file_5 = 'kd15zv05100000.csv'
df_all_labor = pd.read_csv(os.path.join(temp_dir, csv_file_5), dtype='str')
df_all_labor = df_all_labor[df_all_labor['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_labor.shape)

(19818, 70)


In [15]:
df_labor = df_all_labor[[
    '地域ＩＤ',
    '男女別コード',
    '15歳以上人口（人） ',
    '15歳以上人口／労働力人口（人）',
    '15歳以上人口／非労働力人口（人）',
    '15歳以上人口／非労働力人口／家事（人）',       # 専業主婦
    '15歳以上人口／非労働力人口／通学者（人）',     # 高校生・大学生
    '15歳以上人口／非労働力人口／その他（人）',     # 幼児・高齢者
    '(再掲)15歳以上人口／労働力人口／就業者／うち雇用者（人）',     # サラリーマン
    '(再掲)15歳以上人口／労働力人口／就業者／うち役員（人）',       # 役員
    '(再掲)15歳以上人口／労働力人口／就業者／うち自営業主（人）',   # 自営業者
    '(再掲)15歳以上人口／労働力人口／就業者／家事のほか仕事（人）',     # パートタイム
    '(再掲)15歳以上人口／労働力人口／就業者／通学のかたわら仕事（人）', # 学生アルバイト
]].copy()
df_labor.columns = [
    'town_id',
    '男女',
    '15歳以上人口（人）',
    '労働力人口（人）',
    '非労働力人口（人）',
    '非労働力_専業主婦（人）',
    '非労働力_学生（人）',
    '非労働力_幼児・高齢者（人）',
    '労働力_サラリーマン（人）',
    '労働力_役員（人）',
    '労働力_自営業者（人）',
    '労働力_パートタイム（人）',
    '労働力_学生アルバイト（人）',
]
df_labor = df_labor.replace('X', 0).replace('-', 0)
df_labor = df_labor[df_labor['男女'] == '1'].reset_index(drop=True)
labor_cols = [col for col in df_labor.columns if '（人）' in col]
for col in labor_cols:
    df_labor[col] = df_labor[col].astype(int)
print(df_labor.shape)
assert df_labor['town_id'].nunique() == len(df_labor)

(6606, 13)


In [16]:
for col in labor_cols:
    if col != '15歳以上人口（人）':
        new_col = col.replace('（人）', '割合')
        df_labor[new_col] = df_labor[col] / df_labor['15歳以上人口（人）']

df_labor = df_labor.drop(columns=labor_cols+['男女'])
print(df_labor.shape)

(6606, 11)


## 住居の種類

In [17]:
csv_file_6 = 'kd15zv06100000.csv'
df_all_residence = pd.read_csv(os.path.join(temp_dir, csv_file_6), dtype='str')
df_all_residence = df_all_residence[df_all_residence['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_residence.shape)

(6606, 50)


In [18]:
df_residence = df_all_residence[[
    '地域ＩＤ',
    '男女別コード',
    '住宅に住む一般世帯／主世帯／世帯数（世帯）',
    '住宅に住む一般世帯／主世帯／持ち家／世帯数（世帯）',
    '住宅に住む一般世帯／主世帯／公営・都市再生機構・公社の借家／世帯数（世帯）',
    '住宅に住む一般世帯／主世帯／民営の借家／世帯数（世帯）',
    '住宅に住む一般世帯／主世帯／給与住宅／世帯数（世帯）',
]].copy()
df_residence.columns = [
    'town_id',
    '男女',
    '一般世帯_世帯数',
    '持ち家_世帯数',
    '公営借家_世帯数',
    '民営借家_世帯数',
    '社宅_世帯数', 
]
df_residence = df_residence.replace('X', 0).replace('-', 0)
df_residence = df_residence[df_residence['男女'] == '1'].reset_index(drop=True)
residence_cols = [col for col in df_residence.columns if '_世帯数' in col]
for col in residence_cols:
    df_residence[col] = df_residence[col].astype(int)
print(df_residence.shape)
assert df_residence['town_id'].nunique() == len(df_residence)

(6606, 7)


In [19]:
for col in residence_cols:
    if col != '一般世帯_世帯数':
        new_col = col.replace('世帯数', '割合')
        df_residence[new_col] = df_residence[col] / df_residence['一般世帯_世帯数']

df_residence = df_residence.drop(columns=residence_cols+['男女'])
print(df_residence.shape)

(6606, 5)


## 住宅の建て方

In [20]:
csv_file_7 = 'kd15zv07100000.csv'
df_all_building = pd.read_csv(os.path.join(temp_dir, csv_file_7), dtype='str')
df_all_building = df_all_building[df_all_building['オリジナル地域階層コード'].notna()].reset_index(drop=True)
print(df_all_building.shape)

(6606, 65)


In [21]:
df_building = df_all_building[[
    '地域ＩＤ',
    '男女別コード',
    '住宅に住む一般世帯／世帯数（世帯）',
    '住宅に住む一般世帯／一戸建／世帯数（世帯）',
    '住宅に住む一般世帯／長屋建／世帯数（世帯）',
    '住宅に住む一般世帯／共同住宅／１・２階建／世帯数（世帯）',
    '住宅に住む一般世帯／共同住宅／３～５階建／世帯数（世帯）',
    '住宅に住む一般世帯／共同住宅／６～10階建／世帯数（世帯）',
    '住宅に住む一般世帯／共同住宅／11階建以上／世帯数（世帯）',
]].copy()
df_building.columns = [
    'town_id',
    '男女',
    '一般世帯_世帯数',
    '一戸建_世帯数',
    '長屋建_世帯数',
    '共同住宅_１・２階建_世帯数',
    '共同住宅_３～５階建_世帯数',
    '共同住宅_６～10階建_世帯数',
    '共同住宅_11階建以上_世帯数',
]
df_building = df_building.replace('X', 0).replace('-', 0)
df_building = df_building[df_building['男女'] == '1'].reset_index(drop=True)
building_cols = [col for col in df_building.columns if '_世帯数' in col]
for col in building_cols:
    df_building[col] = df_building[col].astype(int)
print(df_building.shape)
assert df_building['town_id'].nunique() == len(df_building)

(6606, 9)


In [22]:
for col in building_cols:
    if col != '一般世帯_世帯数':
        new_col = col.replace('世帯数', '割合')
        df_building[new_col] = df_building[col] / df_building['一般世帯_世帯数']

df_building = df_building.drop(columns=building_cols+['男女'])
print(df_building.shape)

(6606, 7)


## 全てのデータをマージ

In [23]:
df_population_master = pd.merge(df_popdata, df_family, on='town_id', how='left')
df_population_master = pd.merge(df_population_master, df_labor, on='town_id', how='left')
df_population_master = pd.merge(df_population_master, df_residence, on='town_id', how='left')
df_population_master = pd.merge(df_population_master, df_building, on='town_id', how='left')
print(df_population_master.shape)

(3145, 59)


## 出力

In [35]:
df_population_master.to_csv(os.path.join(DIR_NAME, 'population_master.csv'), index=False, encoding='utf_8_sig')

In [30]:
list(df_population_master.columns)

['town_id',
 '市区町村名',
 '町名',
 '町丁目',
 '人口総数（人）',
 '面積（km2）',
 '人口密度(人/km2)',
 '0〜4歳（人）',
 '5〜9歳（人）',
 '10〜14歳（人）',
 '15〜19歳（人）',
 '20〜24歳（人）',
 '25〜29歳（人）',
 '30〜34歳（人）',
 '35〜39歳（人）',
 '40〜44歳（人）',
 '45〜49歳（人）',
 '50〜54歳（人）',
 '55〜59歳（人）',
 '60〜64歳（人）',
 '65〜69歳（人）',
 '70〜74歳（人）',
 '75〜79歳（人）',
 '80〜84歳（人）',
 '85〜89歳（人）',
 '90〜94歳（人）',
 '95〜99歳（人）',
 '100歳以上（人）',
 '年齢不詳（人）',
 '平均年齢（歳）',
 '外国人（人）',
 '世帯総数',
 '1人世帯',
 '2人世帯',
 '3人世帯',
 '4人世帯',
 '5人以上世帯',
 '世帯人員（人）',
 '1世帯当たり人員（人）',
 '労働力人口割合',
 '非労働力人口割合',
 '非労働力_専業主婦割合',
 '非労働力_学生割合',
 '非労働力_幼児・高齢者割合',
 '労働力_サラリーマン割合',
 '労働力_役員割合',
 '労働力_自営業者割合',
 '労働力_パートタイム割合',
 '労働力_学生アルバイト割合',
 '持ち家_割合',
 '公営借家_割合',
 '民営借家_割合',
 '社宅_割合',
 '一戸建_割合',
 '長屋建_割合',
 '共同住宅_１・２階建_割合',
 '共同住宅_３～５階建_割合',
 '共同住宅_６～10階建_割合',
 '共同住宅_11階建以上_割合']

In [34]:
score_family = df_population_master['世帯総数'].quantile(0.2)
score_housewife = df_population_master['非労働力_専業主婦割合'].quantile(0.8)
score_towerapartment = df_population_master['共同住宅_11階建以上_割合'].quantile(0.8)
score_ownedhouse = df_population_master['持ち家_割合'].quantile(0.8)
score_boardmember = df_population_master['労働力_役員割合'].quantile(0.8)
df_population_master[
    (df_population_master['世帯総数'] > score_family) &
    (df_population_master['非労働力_専業主婦割合'] > score_housewife) &
    (df_population_master['共同住宅_11階建以上_割合'] > score_towerapartment) &
    (df_population_master['持ち家_割合'] > score_ownedhouse) &
    (df_population_master['労働力_役員割合'] > score_boardmember)
].sort_values('世帯総数', ascending=False)

,town_id,市区町村名,町名,町丁目,人口総数（人）,面積（km2）,人口密度(人/km2),0〜4歳（人）,5〜9歳（人）,10〜14歳（人）,15〜19歳（人）,20〜24歳（人）,25〜29歳（人）,30〜34歳（人）,35〜39歳（人）,40〜44歳（人）,45〜49歳（人）,50〜54歳（人）,55〜59歳（人）,60〜64歳（人）,65〜69歳（人）,70〜74歳（人）,75〜79歳（人）,80〜84歳（人）,85〜89歳（人）,...,3人世帯,4人世帯,5人以上世帯,世帯人員（人）,1世帯当たり人員（人）,労働力人口割合,非労働力人口割合,非労働力_専業主婦割合,非労働力_学生割合,非労働力_幼児・高齢者割合,労働力_サラリーマン割合,労働力_役員割合,労働力_自営業者割合,労働力_パートタイム割合,労働力_学生アルバイト割合,持ち家_割合,公営借家_割合,民営借家_割合,社宅_割合,一戸建_割合,長屋建_割合,共同住宅_１・２階建_割合,共同住宅_３～５階建_割合,共同住宅_６～10階建_割合,共同住宅_11階建以上_割合
819,108076,江東区,豊洲,豊洲３丁目,7877,0.31,25409.7,872,534,277,195,184,339,838,1211,1156,797,519,295,180,142,80,41,28,13,...,799,510,66,7877,2.287166,0.580399,0.188372,0.125415,0.038040,0.024917,0.422259,0.061960,0.026578,0.054651,0.006478,0.759416,0.000000,0.211971,0.028613,0.000582,0.000000,0.000000,0.000000,0.074505,0.924622
206,102115,中央区,勝どき,勝どき６丁目,6995,0.10,69950.0,517,504,349,212,196,325,540,784,910,778,560,340,281,274,170,130,70,35,...,628,459,81,6995,2.276277,0.549964,0.222617,0.128023,0.038407,0.056188,0.368243,0.074502,0.030050,0.064900,0.007112,0.615686,0.063072,0.296405,0.024837,0.000326,0.000651,0.000000,0.000000,0.079153,0.918893
924,109007,品川区,北品川,北品川５丁目,6320,0.23,27478.3,407,298,198,170,179,328,578,663,666,576,426,316,336,376,233,202,131,79,...,553,325,57,6320,2.060646,0.583270,0.264005,0.136260,0.041635,0.086109,0.418433,0.060939,0.036147,0.059425,0.008138,0.698007,0.000000,0.265781,0.036213,0.072512,0.004944,0.018787,0.145353,0.160514,0.597891
1322,111228,大田区,下丸子,下丸子４丁目,6549,0.52,0.0,248,331,379,237,209,296,295,357,624,630,502,397,387,427,331,282,249,154,...,529,409,76,6316,2.169701,0.524510,0.374183,0.164488,0.051017,0.158678,0.388344,0.052106,0.034314,0.067720,0.007081,0.655715,0.011884,0.307585,0.024816,0.150970,0.007964,0.076177,0.242382,0.025970,0.495152
1642,113011,渋谷区,広尾,広尾４丁目,4599,0.25,18396.0,283,169,143,236,247,172,187,270,343,304,290,233,238,329,231,180,183,174,...,321,176,43,3929,2.024214,0.388992,0.377559,0.131880,0.080298,0.165382,0.197554,0.096783,0.032172,0.051050,0.016751,0.794521,0.000000,0.105374,0.100105,0.000525,0.000000,0.000000,0.018373,0.395276,0.585827
317,103130,港区,白金台,白金台３丁目,3668,0.16,22925.0,171,213,170,137,102,135,225,310,335,341,284,232,200,258,169,140,119,74,...,288,212,59,3668,2.105626,0.427926,0.279587,0.137375,0.047082,0.095131,0.228958,0.067398,0.046437,0.062560,0.004837,0.692441,0.000000,0.278130,0.029429,0.158137,0.001725,0.037378,0.278896,0.182864,0.341001
1509,112190,世田谷区,玉川,玉川１丁目,3443,0.30,11476.7,236,190,107,122,96,106,223,328,360,321,269,184,168,200,135,95,61,37,...,327,196,27,3443,2.139838,0.471906,0.257437,0.143224,0.042233,0.071979,0.300771,0.065002,0.031950,0.049578,0.007345,0.751412,0.000000,0.241055,0.007533,0.087796,0.008095,0.061021,0.214197,0.059153,0.569116
818,108075,江東区,豊洲,豊洲２丁目,3671,0.28,13110.7,287,325,181,99,77,96,190,405,531,423,252,185,151,158,100,60,38,9,...,328,283,45,3671,2.415132,0.543557,0.258099,0.150468,0.038517,0.069114,0.397768,0.058315,0.021598,0.055436,0.007919,0.927767,0.000000,0.055666,0.016567,0.000000,0.000000,0.000000,0.000000,0.026403,0.972937
786,108031,江東区,白河,白河２丁目,2478,0.08,30975.0,99,144,124,72,97,129,157,213,289,258,232,128,115,141,86,67,68,43,...,202,145,32,2477,2.102716,0.604453,0.257224,0.125059,0.040265,0.091900,0.449076,0.051161,0.037897,0.062056,0.007579,0.623377,0.000000,0.349784,0.026840,0.244236,0.003416,0.024765,0.137489,0.162254,0.422716
1709,113095,渋谷区,神宮前,神宮前１丁目,1933,0.22,8786.4,82,66,84,82,85,109,102,122,178,195,178,122,75,67,51,56,26,29,...,137,105,19,1933,1.884016,0.524064,0.271390,0.132353,0.072193,0.066845,0.336898,0.079545,0.053476,0.067513,0.005348,0.613419,0.000000,0.172524,0.214058,0.088421,0.003158,0.004211,0.084211,0.453684,0.356842


In [27]:
df_population_master[
    (df_population_master['市区町村名'] == '港区') & 
    (df_population_master['町名'] == '三田')
]

,town_id,市区町村名,町名,町丁目,人口総数（人）,面積（km2）,人口密度(人/km2),0〜4歳（人）,5〜9歳（人）,10〜14歳（人）,15〜19歳（人）,20〜24歳（人）,25〜29歳（人）,30〜34歳（人）,35〜39歳（人）,40〜44歳（人）,45〜49歳（人）,50〜54歳（人）,55〜59歳（人）,60〜64歳（人）,65〜69歳（人）,70〜74歳（人）,75〜79歳（人）,80〜84歳（人）,85〜89歳（人）,...,3人世帯,4人世帯,5人以上世帯,世帯人員（人）,1世帯当たり人員（人）,労働力人口割合,非労働力人口割合,非労働力_専業主婦割合,非労働力_学生割合,非労働力_幼児・高齢者割合,労働力_サラリーマン割合,労働力_役員割合,労働力_自営業者割合,労働力_パートタイム割合,労働力_学生アルバイト割合,持ち家_割合,公営借家_割合,民営借家_割合,社宅_割合,一戸建_割合,長屋建_割合,共同住宅_１・２階建_割合,共同住宅_３～５階建_割合,共同住宅_６～10階建_割合,共同住宅_11階建以上_割合
233,103027,港区,三田,三田１丁目,4248,0.22,19309.1,239,157,129,128,167,306,405,464,470,449,332,197,150,173,161,132,113,45,...,332,179,42,4248,1.803822,0.440892,0.200967,0.092423,0.033853,0.074691,0.234820,0.060451,0.027405,0.042988,0.004567,0.625650,0.024697,0.308059,0.041594,0.104766,0.006441,0.028768,0.088450,0.277372,0.492915
234,103028,港区,三田,三田２丁目,4025,0.30,13416.7,204,140,131,138,145,256,335,360,405,350,293,240,200,262,191,153,112,72,...,352,171,34,4025,1.920324,0.435984,0.213480,0.110829,0.035815,0.066836,0.234913,0.070784,0.034123,0.042019,0.004512,0.651062,0.012066,0.290541,0.046332,0.098801,0.000959,0.014868,0.194724,0.303118,0.386571
235,103029,港区,三田,三田３丁目,1534,0.16,9587.5,65,43,32,67,105,127,131,160,143,108,109,86,68,88,67,47,38,34,...,97,46,17,1460,1.638608,0.404454,0.198276,0.063937,0.064655,0.069684,0.220546,0.053161,0.022989,0.035920,0.014368,0.440181,0.000000,0.513544,0.046275,0.080899,0.000000,0.012360,0.183146,0.230337,0.478652
236,103030,港区,三田,三田４丁目,3400,0.24,14166.7,165,113,110,111,125,162,255,319,331,294,234,207,181,253,185,146,100,64,...,253,155,42,3400,1.909040,0.435983,0.238444,0.107749,0.038244,0.092451,0.263053,0.058863,0.036581,0.051879,0.006319,0.655840,0.000000,0.319658,0.024501,0.161108,0.003957,0.033917,0.258903,0.302996,0.235726
237,103031,港区,三田,三田５丁目,5296,0.11,48145.5,227,203,193,192,215,334,437,537,518,517,419,308,275,303,204,157,129,86,...,409,229,62,5296,1.871378,0.468008,0.219345,0.104002,0.041301,0.074042,0.288038,0.050503,0.038305,0.052643,0.006206,0.601140,0.004986,0.361467,0.032407,0.075231,0.001774,0.009581,0.086586,0.222853,0.601136
